In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np
import re
from datetime import datetime

In [ ]:
gp=pd.read_csv("/Users/gevorgkhandamiryan/Desktop/nhsdata/GP_BRANCHES.csv")
gp.columns=['web-scraper-order', 'web-scraper-start-url', 'name', 'name-href',
       'address','reviewlink', 'reviewlink-href', 'summary', 'stars', 'reviewer',
       'bodytext', 'datevisit', 'replier', 'replydate', 'replytext']
gp

In [ ]:
hcare=pd.read_csv("/Users/gevorgkhandamiryan/Desktop/nhsdata/HCARE.csv")
hcare.columns=['web-scraper-order', 'web-scraper-start-url', 'name', 'name-href',
       'reviewlink', 'reviewlink-href', 'summary',
       'reviewer', 'stars','bodytext', 'datevisit', 'replier', 'replydate',
       'replytext','address']
hcare

In [ ]:
mental=pd.read_csv("/Users/gevorgkhandamiryan/Desktop/nhsdata/MENTAL.csv")
mental.columns=['web-scraper-order', 'web-scraper-start-url', 'name', 'name-href',
        'reviewlink', 'reviewlink-href', 'summary', 
       'reviewer','stars', 'bodytext', 'datevisit', 'replier', 'replydate',
       'replytext','address']
mental

In [ ]:
acute=pd.read_csv("/Users/gevorgkhandamiryan/Desktop/nhsdata/ACUTE.csv")
acute

In [ ]:
def cleaning(nhs):
    
    nhsnew=nhs.drop(columns={'web-scraper-order','web-scraper-start-url','name-href','reviewlink', 'reviewlink-href'})
    # CLEANING STARS
    # FILLING IN NAN VALUES
    num_stars=nhsnew['stars'].fillna(' ')
    i=0
    list1=[]
    while i<sum(num_stars.notnull()):
        list1.append(num_stars[i].split(' out')[0].count('★'))
        i=i+1
    nhsnew['stars']=list1
    # CLEANING REVIEW TEXT
    # NUMBER OF REVIEWS THAT ARE NOT NULL
    sum(nhsnew['bodytext'].notnull())
    # FILLING IN NAN VALUES
    nhsnew['bodytext']=nhsnew['bodytext'].fillna(' ')
    nhsnew['bodytext']=nhsnew['bodytext'].str.replace('\n',' ').str.replace('\t',' ').str.replace('\r',' ')
    # SENTIMENT FOR TEXTUAL ANALYSIS
    reviewtext=nhsnew['bodytext'].str.split(expand=True).stack().reset_index(level=1).rename(columns={'level_1': 'num', 0: 'word'})
    sent = pd.read_csv('vader_lexicon.txt', sep='\t',
                   usecols=[0, 1], header=None, names=['token', 'polarity'],
                   index_col='token')
    # CALCULATING POLARITIES
    nhsnew['polarity'] = (
        reviewtext
        .merge(sent, how='left', left_on='word', right_index=True)
        .reset_index()
        .loc[:, ['index', 'polarity']]
        .groupby('index')
        .sum()
        .fillna(0)
    )
    # CALCULATING REVIEW LENGTHS
    nhsnew['len'] = (
        reviewtext
        .merge(sent, how='left', left_on='word', right_index=True)
        .reset_index()
        .groupby('index')
        .max()
        .fillna(0)
        ['num']
    )
    # CLEANING DATE OF VISIT
    visits=nhsnew['datevisit']
    visits=visits.fillna('Visited 0').str.split('ed ')
    visits=[i[1] for i in visits]
    visits=pd.DataFrame(visits)
    visit=[i[:3]+ i[-4:] for i in visits[0]]
    visit=pd.DataFrame(visit)
    visit = pd.to_datetime(visit[0], format='%b%Y',errors='coerce')
    clean=visit
    nhsnew['datevisit']=clean
    # CLEANING ADDRESS
    nhsnew['address']=nhsnew['address'].str.replace('\n',' ').str.replace('\t',' ').str.replace('\r',' ')
    # CLEANING REVIEWER COLUMN    
    reviewer=nhsnew['reviewer']
    revclean=reviewer.str.split(' - Posted on ',expand=True)
    revperson=[x[3:] for x in revclean[0].fillna('by NaN')]
    revdate=revclean[[1]].fillna('50 January 2090')
    revdate=[i[:2]+ i[3:6]+ i[-4:] for i in revdate[1]]
    revdate=pd.DataFrame(revdate)
    revdate = pd.to_datetime(revdate[0], format='%d%b%Y',errors='coerce')
    a=reviewer
    nhsnew['reviewer']=revperson
    nhsnew['datereview']=revdate
    # CLEANING DATE OF REPLY
    repdate=nhsnew['replydate']
    repdate=repdate.fillna('Replied on 0').str.split('Replied on')
    repdate=[i[1] for i in repdate]
    repdate=pd.DataFrame(repdate)
    repdate=[i[1:3]+ i[4:7]+ i[-4:] for i in repdate[0]]
    repdate=pd.DataFrame(repdate)
    repdate = pd.to_datetime(repdate[0], format='%d%b%Y',errors='coerce')
    a=repdate
    nhsnew['replydate']=a
    # CLEANING REPLIER COLUMN
    nhsnew['replier']=nhsnew['replier'].str.replace('\n',' ').str.replace('\t',' ').str.replace('\r',' ')
    a=nhsnew['replier'].fillna('Review titled NaN').str.split('titled          ',expand=True)[1]
    nhsnew['replier']=a
    # CLEANING SUMMARY COLUMN  
    nhsnew['summary']=nhsnew['summary'].str.replace('\n',' ').str.replace('\t',' ').str.replace('\r',' ')
    b=nhsnew['summary'].fillna('Review titled NaN').str.split('titled          ',expand=True)[1]
    nhsnew['summary']=b
    # CLEANING REPLY TEXT COLUMN      
    nhsnew['replytext']=nhsnew['replytext'].str.replace('\n',' ').str.replace('\t',' ').str.replace('\r',' ')
    # SENTIMENT ANALYSIS FOR SUMMARY AND REPLY TEXT
    replytext=nhsnew['replytext'].str.split(expand=True).stack().reset_index(level=1).rename(columns={'level_1': 'num', 0: 'word'})
    sumtext=nhsnew['summary'].str.split(expand=True).stack().reset_index(level=1).rename(columns={'level_1': 'num', 0: 'word'})
    nhsnew['replypolarity'] = (
    replytext
        .merge(sent, how='left', left_on='word', right_index=True)
        .reset_index()
        .loc[:, ['index', 'polarity']]
        .groupby('index')
        .sum()
        .fillna(0)
    )
    nhsnew['sumpolarity'] = (
        sumtext
        .merge(sent, how='left', left_on='word', right_index=True)
        .reset_index()
        .loc[:, ['index', 'polarity']]
        .groupby('index')
        .sum()
        .fillna(0)
    )
    nhsnew['avgreplypolarity'] = (
        replytext
        .merge(sent, how='left', left_on='word', right_index=True)
        .reset_index()
        .loc[:, ['index', 'polarity']]
        .groupby('index')
        .mean()
        .fillna(0)
        )
    nhsnew['avgpolarity'] = (
        reviewtext
        .merge(sent, how='left', left_on='word', right_index=True)
        .reset_index()
        .loc[:, ['index', 'polarity']]
        .groupby('index')
        .mean()
        .fillna(0)
        )
    nhsnew['avgsumpolarity'] = (
        sumtext
        .merge(sent, how='left', left_on='word', right_index=True)
        .reset_index()
        .loc[:, ['index', 'polarity']]
        .groupby('index')
        .mean()
        .fillna(0)
        )
    nhsnew['responsiveness']=[x.days for x in nhsnew['replydate']-nhsnew['datereview'] ]
    j=0
    dummy=[]
    while j<sum(num_stars.notnull()): 
        if nhsnew['replier'][j]==nhsnew['name'][j]:
            dummy.append(1)
        else:
            dummy.append(0)
        j+=1
    nhsnew['dummy']=dummy
    k=0
    dummy2=[]
    while k<sum(num_stars.notnull()): 
        if nhsnew['reviewer'][k]=='Anonymous':
            dummy2.append(0)
        else:
            dummy2.append(1)
        k+=1
    nhsnew['dummy2']=dummy2
    
    return nhsnew

In [ ]:
data1=cleaning(gp)
data2=cleaning(acute)
data3=cleaning(mental)
data4=cleaning(hcare)

In [ ]:
import geopandas as gpd
shapefile = "/Users/gevorgkhandamiryan/Desktop/Distribution/Districts.shp"
#Read shapefile using Geopandas
gdf = gpd.read_file(shapefile)

gdf.columns=['postcode','geometry']

In [ ]:
data1['postcodes']=data1['address'].str.extract(r"([A-Z]{1,}[0-9]{1,})")

merged1=pd.merge(gdf, data1, right_on='postcodes',left_on='postcode',how='inner')
merged1=merged1[['postcode', 'geometry', 'name', 'address', 'summary', 'stars',
       'reviewer', 'bodytext', 'replier', 
       'replytext', 'polarity', 'len', 'replypolarity',
       'sumpolarity', 'avgreplypolarity', 'avgpolarity', 'avgsumpolarity',
       'responsiveness', 'dummy', 'dummy2', 'postcodes']]

data2['postcodes']=data2['address'].str.extract(r"([A-Z]{1,}[0-9]{1,})")

merged2=pd.merge(gdf, data2, right_on='postcodes',left_on='postcode',how='inner')
merged2=merged2[['postcode', 'geometry', 'name', 'address', 'summary', 'stars',
       'reviewer', 'bodytext', 'replier', 
       'replytext', 'polarity', 'len', 'replypolarity',
       'sumpolarity', 'avgreplypolarity', 'avgpolarity', 'avgsumpolarity',
       'responsiveness', 'dummy', 'dummy2', 'postcodes']]

data3['postcodes']=data3['address'].str.extract(r"([A-Z]{1,}[0-9]{1,})")

merged3=pd.merge(gdf, data3, right_on='postcodes',left_on='postcode',how='inner')
merged3=merged3[['postcode', 'geometry', 'name', 'address', 'summary', 'stars',
       'reviewer', 'bodytext', 'replier', 
       'replytext', 'polarity', 'len', 'replypolarity',
       'sumpolarity', 'avgreplypolarity', 'avgpolarity', 'avgsumpolarity',
       'responsiveness', 'dummy', 'dummy2', 'postcodes']]

data4['postcodes']=data4['address'].str.extract(r"([A-Z]{1,}[0-9]{1,})")

merged4=pd.merge(gdf, data4, right_on='postcodes',left_on='postcode',how='inner')
merged4=merged4[['postcode', 'geometry', 'name', 'address', 'summary', 'stars',
       'reviewer', 'bodytext', 'replier', 
       'replytext', 'polarity', 'len', 'replypolarity',
       'sumpolarity', 'avgreplypolarity', 'avgpolarity', 'avgsumpolarity',
       'responsiveness', 'dummy', 'dummy2', 'postcodes']]

In [ ]:
import json
#Read data to json.
merged_json = json.loads(merged1.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)




from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer




#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)
#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 5)
#Define custom tick labels for color bar.
tick_labels = {'0': '0', '1': '1', '2':'2', '3':'3', '4':'4', '5':'5'}
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = 'GP Branches', plot_height = 1300 , plot_width = 950, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'stars', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)



In [ ]:
merged2=pd.merge(gdf, data2, right_on='postcodes',left_on='postcode',how='inner')
merged2=merged2[['postcode', 'geometry', 'name', 'address', 'summary', 'stars',
       'reviewer', 'bodytext', 'replier', 
       'replytext', 'polarity', 'len', 'replypolarity',
       'sumpolarity', 'avgreplypolarity', 'avgpolarity', 'avgsumpolarity',
       'responsiveness', 'dummy', 'dummy2', 'postcodes']]

import json
#Read data to json.
merged_json = json.loads(merged2.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)




from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer




#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)
#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 5)
#Define custom tick labels for color bar.
tick_labels = {'0': '0', '1': '1', '2':'2', '3':'3', '4':'4', '5':'5'}
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = 'Acute Trusts', plot_height = 1300 , plot_width = 950, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'stars', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

In [ ]:
merged3=pd.merge(gdf, data3, right_on='postcodes',left_on='postcode',how='inner')
merged3=merged3[['postcode', 'geometry', 'name', 'address', 'summary', 'stars',
       'reviewer', 'bodytext', 'replier', 
       'replytext', 'polarity', 'len', 'replypolarity',
       'sumpolarity', 'avgreplypolarity', 'avgpolarity', 'avgsumpolarity',
       'responsiveness', 'dummy', 'dummy2', 'postcodes']]

import json
#Read data to json.
merged_json = json.loads(merged3.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)




from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer




#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)
#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 5)
#Define custom tick labels for color bar.
tick_labels = {'0': '0', '1': '1', '2':'2', '3':'3', '4':'4', '5':'5'}
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = 'Share of adults who are obese, 2016', plot_height = 1300 , plot_width = 950, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'stars', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

In [ ]:
merged4=pd.merge(gdf, data4, right_on='postcodes',left_on='postcode',how='inner')
merged4=merged4[['postcode', 'geometry', 'name', 'address', 'summary', 'stars',
       'reviewer', 'bodytext', 'replier', 
       'replytext', 'polarity', 'len', 'replypolarity',
       'sumpolarity', 'avgreplypolarity', 'avgpolarity', 'avgsumpolarity',
       'responsiveness', 'dummy', 'dummy2', 'postcodes']]

import json
#Read data to json.
merged_json = json.loads(merged4.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)




from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer




#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)
#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 5)
#Define custom tick labels for color bar.
tick_labels = {'0': '0', '1': '1', '2':'2', '3':'3', '4':'4', '5':'5'}
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = 'Share of adults who are obese, 2016', plot_height = 1300 , plot_width = 950, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'stars', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

In [ ]:
data1['type']='gp'
data2['type']='acute'
data3['type']='mental'
data4['type']='hcare'
data=pd.concat([data1,data2,data3,data4])
dum=pd.get_dummies(data['type'])
final=pd.concat([data,dum],axis=1)

In [ ]:
data1['len_bin'] = pd.cut(data1['len'], np.arange(0,750,150))
sns.relplot('stars','polarity', data=data1, hue='len_bin')
plt.title('GP branches')
plt.show()
data2['len_bin'] = pd.cut(data2['len'], np.arange(0,750,150))
sns.relplot('stars','polarity', data=data2, hue='len_bin')
plt.title('Acute Trusts')
plt.show()
data3['len_bin'] = pd.cut(data3['len'], np.arange(0,750,150))
sns.relplot('stars','polarity', data=data3, hue='len_bin')
plt.title('Mental Health Trusts')
plt.show()
data4['len_bin'] = pd.cut(data4['len'], np.arange(0,750,150))
sns.relplot('stars','polarity', data=data4, hue='len_bin')
plt.title('Health and Care Trusts')
plt.show()

In [ ]:
plt.figure(figsize=(15,8))

sns.lineplot(x='datereview',y='stars',data=data1)
plt.title('GP branches')
plt.show()

plt.figure(figsize=(15,8))
sns.lineplot(x='datereview',y='stars',data=data2)
plt.title('Acute Trusts')
plt.show()

plt.figure(figsize=(15,8))
sns.lineplot(x='datereview',y='stars',data=data3)
plt.title('Mental Health Trusts')
plt.show()

plt.figure(figsize=(15,8))
sns.lineplot(x='datereview',y='stars',data=data4)
plt.title('Health and Care Trusts')
plt.show()

In [ ]:
data1.describe()

In [ ]:
data2.describe()

In [ ]:
data3.describe()

In [ ]:
data4.describe()

In [ ]:
plt.figure(figsize=(15,8))

sns.lineplot(x='datereview',y='dummy',data=data1)
plt.title('GP branches')
plt.ylabel('Probability of replying')
plt.show()

plt.figure(figsize=(15,8))
sns.lineplot(x='datereview',y='dummy',data=data2)
plt.title('Acute Trusts')
plt.show()

plt.figure(figsize=(15,8))
sns.lineplot(x='datereview',y='dummy',data=data3)
plt.title('Mental Health Trusts')
plt.show()

plt.figure(figsize=(15,8))
sns.lineplot(x='datereview',y='dummy',data=data4)
plt.title('Health and Care Trusts')
plt.show()

In [ ]:
import json
#Read data to json.
merged_json = json.loads(merged1.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)




from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer




#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)
#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 50)
#Define custom tick labels for color bar.
tick_labels = {'0': '0', '1': '10', '2':'20', '3':'30', '4':'40', '5':'50'}
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = 'GP Branches', plot_height = 1300 , plot_width = 950, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'responsiveness', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)



In [ ]:
import json
#Read data to json.
merged_json = json.loads(merged2.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)




from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer




#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)
#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 50)
#Define custom tick labels for color bar.
tick_labels = {'0': '0', '1': '10', '2':'20', '3':'30', '4':'40', '5':'50'}
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = 'Acute Trusts', plot_height = 1300 , plot_width = 950, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'responsiveness', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)




In [ ]:
final.to_csv('nhsfinal.csv')